### CubiCasa5k dataset + code reproduction

The dataset has some `.png` files and some `.svg` files.
There is the official `cubicasa5k`git repo, where they give the code to load the dataset!

In [ ]:
%%capture
%pip install cairosvg svgpathtools lmdb shapely
%conda install scikit-image -y

In [ ]:
import sys
from io import BytesIO
from pathlib import Path

import cairosvg
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from IPython.display import HTML, display
from IPython.display import Image as IPImage
from pdf2image import convert_from_path
from PIL import Image

BASE_FOLDER = Path("/ifc_dl")

sys.path.append(str(BASE_FOLDER / "submodules/CubiCasa5k"))

from floortrans.loaders import FloorplanSVG
from floortrans.models import get_model
from floortrans.plotting import (
    discrete_cmap,
    draw_junction_from_dict,
    polygons_to_image,
)
from floortrans.post_prosessing import get_polygons, split_prediction
from nb_utils.cubicasa5k_utils import icon_classes, predict_with_rotations, room_classes
from nb_utils.utils import EcoDatasetTiled512, rmtree

try:
    discrete_cmap()
except ValueError as e:
    if 'colormap named "rooms"' in str(e):
        pass

cubicasa5k_path = BASE_FOLDER / "data/cubicasa5k"

In [ ]:
example_folder = BASE_FOLDER / "data/cubicasa5k/colorful/34/"
svf_file = example_folder / "model.svg"
png_file = example_folder / "F1_original.png"

# png file reading
image = Image.open(png_file).convert("RGB")
png_array = np.array(image)

# svg file reading
with open(str(svf_file), "r") as file:
    svg_data = file.read()

png_data = cairosvg.svg2png(bytestring=svg_data)
image = Image.open(BytesIO(png_data))
svg_array = np.array(image)

# plot the png and svg files
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("SVG")
plt.imshow(svg_array)
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("PNG")
plt.imshow(png_array)
plt.axis("off")

plt.show()

#### Reading the files with the code from the [original repo](https://github.com/CubiCasa/CubiCasa5k/tree/master).

Took most of the code from `samples.ipynb`.

In [ ]:
data_file = "train.txt"

dataset = FloorplanSVG(
    str(cubicasa5k_path) + "/", data_file, format="txt", original_size=True
)


In [ ]:
# choose a sample from the dataset
sample = dataset[0]


def get_img_as_np(x):
    return np.moveaxis(x.cpu().data.numpy(), 0, -1) / 2 + 0.5


image = sample["image"]
image_np = get_img_as_np(image[0])

label = sample["label"]
label1_np = get_img_as_np(label[0])
label2_np = get_img_as_np(label[1])

scale = sample["scale"]
junctions = sample["heatmaps"]

print(f"The type of the sample: {type(sample)}")
print(f"The keys of the sample: {sample.keys()}")

print(f"\nThe image shape: {image.shape}")
print(f"The mask shape: {label.shape}")
print(f"The scale: {scale}")
print(f"The heatmaps (they call them junctions) are:\n {junctions}")


In [ ]:
# svg model
svg_img = IPImage(url="/ifc_dl/data/cubicasa5k/high_quality/19/model.svg", width=800)
title = "<h3>Plot of the SVG image.</h3>"

# Displaying the title followed by the image
display(HTML(title))
display(svg_img)

# junctions corners and openings
plt.figure(figsize=(6, 4))
plt.imshow(image_np.transpose(1, 0))
plt.title("Wall junctions, Icon corners and opening end points", fontsize=12)
w, h = image_np.shape
draw_junction_from_dict(junctions, w, h, size=0.3, fontsize=10)
plt.axis("off")
plt.show()

# img and segmentation labels
plt.figure(figsize=(12, 8))

plt.subplot(1, 3, 1)
plt.title("Image")
plt.imshow(image_np, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Mask 1")
plt.imshow(label1_np)
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Mask 2")
plt.imshow(label2_np)
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
save_path = BASE_FOLDER / "notebooks/weights/"
save_path.mkdir(exist_ok=True)

!gdown -c 'https://drive.google.com/uc?export=download&id=1gRB7ez1e4H7a9Y09lLqRuna0luZO5VRK' -O /ifc_dl/notebooks/weights/

In [ ]:
%%capture

data_folder = "data/cubicasa5k/"
data_file = "test.txt"

# ############ MODEL SETUP ################

model = get_model("hg_furukawa_original", 51)

n_classes = 44

# this is because there are 44 classes in the dataset, and they have to be split into 3 groups
split = [21, 12, 11]

model.conv4_ = torch.nn.Conv2d(256, n_classes, bias=True, kernel_size=1)
model.upsample = torch.nn.ConvTranspose2d(n_classes, n_classes, kernel_size=4, stride=4)

checkpoint = torch.load(
    "weights/model_best_val_loss_var.pkl", map_location=torch.device("cpu")
)

model.load_state_dict(checkpoint["model_state"])
model.eval()

In [ ]:
if True:
    plt.figure(figsize=(8, 8))
    ax = plt.subplot(1, 1, 1)
    plt.title("Rooms and walls", fontsize=20)
    ax.axis("off")
    n_rooms = 12
    rseg = ax.imshow(label1_np, cmap="rooms", vmin=0, vmax=n_rooms - 0.1)
    cbar = plt.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01)
    cbar.ax.set_yticklabels(room_classes, fontsize=20)
    plt.show()

    plt.figure(figsize=(8, 8))
    ax = plt.subplot(1, 1, 1)
    plt.title("Icons", fontsize=20)
    ax.axis("off")
    n_icons = 11
    iseg = ax.imshow(label2_np, cmap="icons", vmin=0, vmax=n_icons - 0.1)
    cbar = plt.colorbar(iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01)
    cbar.ax.set_yticklabels(icon_classes, fontsize=20)
    plt.show()

    plt.figure(figsize=(8, 8))
    ax = plt.subplot(1, 1, 1)
    plt.title("Wall junctions, Icon corners and opening end points", fontsize=20)
    ax.axis("off")
    ax.imshow(image_np.transpose(1, 0))
    w, h = image_np.shape
    draw_junction_from_dict(junctions, w, h, size=0.3, fontsize=10)
    plt.show()

## Getting network predictions

In [ ]:
def resize_array(label_np, img_shape):
    return F.interpolate(
        torch.from_numpy(label_np)[None, None],
        size=img_shape,
        mode="nearest",
    ).numpy()[0, 0]


ratio = image_np.shape[0] / image_np.shape[1]
desired_width = 512

img_size = (int(desired_width * ratio), desired_width)

height, width = img_size

resized_image = resize_array(image_np, img_size)
resized_label1 = resize_array(label1_np, img_size)
resized_label2 = resize_array(label2_np, img_size)

input_tensor = torch.from_numpy(resized_image)[None, None].repeat(1, 3, 1, 1).float()

prediction = predict_with_rotations(input_tensor, model, n_classes)

In [ ]:
rooms_pred = F.softmax(prediction[0, 21 : 21 + 12], 0).cpu().data.numpy()
rooms_pred = np.argmax(rooms_pred, axis=0)

icons_pred = F.softmax(prediction[0, 21 + 12 :], 0).cpu().data.numpy()
icons_pred = np.argmax(icons_pred, axis=0)

plt.figure(figsize=(6, 6))
ax = plt.subplot(1, 1, 1)
ax.axis("off")
rseg = ax.imshow(rooms_pred, cmap="rooms", vmin=0, vmax=n_rooms - 0.1)
cbar = plt.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(room_classes, fontsize=10)
plt.show()

plt.figure(figsize=(6, 6))
ax = plt.subplot(1, 1, 1)
ax.axis("off")
iseg = ax.imshow(icons_pred, cmap="icons", vmin=0, vmax=n_icons - 0.1)
cbar = plt.colorbar(iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(icon_classes, fontsize=10)
plt.show()


In [ ]:
THRESHOLD = 0.05

heatmaps, rooms, icons = split_prediction(prediction, img_size, split)

polygons, types, room_polygons, room_types = get_polygons(
    (heatmaps, rooms, icons), THRESHOLD, [1, 2]
)

pol_room_seg, pol_icon_seg = polygons_to_image(
    polygons, types, room_polygons, room_types, height, width
)

# ########### PLOTTING ################
plt.figure(figsize=(8, 8))
ax1 = plt.subplot(1, 2, 1)
ax1.axis("off")
ax1.imshow(resized_image)

ax2 = plt.subplot(1, 2, 2)
ax2.axis("off")
rseg = ax2.imshow(pol_room_seg, cmap="rooms", vmin=0, vmax=n_rooms - 0.1)
cbar = plt.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(room_classes, fontsize=20)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 8))
ax1 = plt.subplot(1, 1, 1)
ax1.axis("off")
iseg = ax1.imshow(pol_icon_seg, cmap="icons", vmin=0, vmax=n_icons - 0.1)
cbar = plt.colorbar(iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(icon_classes, fontsize=20)
plt.tight_layout()
plt.show()


## Get predictions on the Ecolution data

In [ ]:
ecolution_data_path = BASE_FOLDER / "/data/ecolution"
floorplans = [x for x in ecolution_data_path.glob("**/*.pdf") if "floorplan" in str(x)]

unique_folder_names = {x.parts[-5] for x in floorplans}
floorplans_files_per_folder = {
    folder: [x for x in floorplans if folder in str(x)]
    for folder in unique_folder_names
}

# img_sample_path = floorplans_files_per_folder["single-family-house"][8]
img_sample_path = "/ifc_dl/data/ecolution/single-family-house/baeumliweg_0_5325_leibstadt/raw/floorplan/0.pdf"

floorplan_sample = np.array(convert_from_path(img_sample_path, dpi=50)[0])
ratio = floorplan_sample.shape[0] / floorplan_sample.shape[1]
img_size = (int(1256 * ratio), 1256)

height, width = img_size

floorplan_sample = resize_array(floorplan_sample[..., 0], img_size)
floorplan_tensor = torch.tensor(floorplan_sample)[None, None].repeat(1, 3, 1, 1) / 255.0

print(f"The shape of the floorplan tensor: {floorplan_tensor.shape}")

# ############# PREDICTION ###############
prediction = predict_with_rotations(floorplan_tensor, model, n_classes)

rooms_pred = F.softmax(prediction[0, 21 : 21 + 12], 0).cpu().data.numpy()
rooms_pred = np.argmax(rooms_pred, axis=0)

icons_pred = F.softmax(prediction[0, 21 + 12 :], 0).cpu().data.numpy()
icons_pred = np.argmax(icons_pred, axis=0)

heatmaps, rooms, icons = split_prediction(prediction, img_size, split)

THRESHOLD = 0.2

polygons, types, room_polygons, room_types = get_polygons(
    (heatmaps, rooms, icons), THRESHOLD, [1, 2]
)

pol_room_seg, pol_icon_seg = polygons_to_image(
    polygons, types, room_polygons, room_types, height, width
)

# ########### PLOTTING ################
plt.figure(figsize=(6, 6))
plt.imshow(np.repeat(floorplan_sample[..., None], 3, axis=-1))
plt.axis("off")
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)
ax.axis("off")
rseg = ax.imshow(rooms_pred, cmap="rooms", vmin=0, vmax=n_rooms - 0.1)
cbar = plt.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(room_classes, fontsize=12)
plt.show()

plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)
ax.axis("off")
iseg = ax.imshow(icons_pred, cmap="icons", vmin=0, vmax=n_icons - 0.1)
cbar = plt.colorbar(iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(icon_classes, fontsize=12)
plt.show()


In [ ]:
# ########### PLOTTING ################

# plot the image by itself
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.imshow(np.repeat(floorplan_sample[..., None], 3, axis=-1))
plt.show()

# plot the prediction and the overlay with the image
plt.figure(figsize=(12, 5))

ax1 = plt.subplot(1, 2, 1)
ax1.set_title("Rooms predictions")
rseg = ax1.imshow(pol_room_seg, cmap="rooms", vmin=0, vmax=n_rooms - 0.1)
cbar = plt.colorbar(rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(room_classes, fontsize=12)
plt.axis("off")

ax2 = plt.subplot(1, 2, 2)
ax2.set_title("Icons predictions")
iseg = ax2.imshow(pol_icon_seg, cmap="icons", vmin=0, vmax=n_icons - 0.1)
cbar = plt.colorbar(iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01)
cbar.ax.set_yticklabels(icon_classes, fontsize=12)
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
%%capture
%pip install fpdf

In [ ]:
building_prototype_path = BASE_FOLDER / "data/building_prototypes"
cropped_paths = list(building_prototype_path.glob("**/**/*crop*"))
cropped_512 = EcoDatasetTiled512(cropped_paths)

In [ ]:
%load_ext autoreload
%autoreload 2

from fpdf import FPDF
from tqdm import tqdm

THRESHOLD = 0.05

building_prototype_path = BASE_FOLDER / "/data/building_prototypes"


pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=5)
page_width = pdf.w - 5

# create a tmp folder to store the plots
tmp_folder = BASE_FOLDER / "notebooks/tmp"
tmp_folder.mkdir(exist_ok=True)

tqdm_bar = tqdm(cropped_512)
for all_tiles, floorplan_path in tqdm_bar:
    floorplan_folder_name = floorplan_path.parts[-2]
    for i, tile in enumerate(all_tiles):
        pdf.add_page()
        pdf.set_font("Arial", style="B", size=16)
        pdf.cell(
            200,
            10,
            txt=f"Folder: {str(floorplan_folder_name).encode('latin-1', 'replace').decode('latin-1')}",
            ln=True,
            align="C",
        )
        pdf.set_font("Arial", size=8)
        pdf.cell(
            200,
            10,
            txt=f"Floorplan: {str(floorplan_path).encode('latin-1', 'replace').decode('latin-1')}",
            ln=True,
            align="C",
        )

        # ############# PREDICTION ###############
        prediction = predict_with_rotations(tile[None], model, n_classes)

        heatmaps, rooms, icons = split_prediction(prediction, img_size, split)

        polygons, types, room_polygons, room_types = get_polygons(
            (heatmaps, rooms, icons), THRESHOLD, [1, 2]
        )

        pol_room_seg, pol_icon_seg = polygons_to_image(
            polygons, types, room_polygons, room_types, height, width
        )

        # ########### PLOTTING ################

        # ------------ 1st plot ------------
        buf1 = BytesIO()
        plt.figure(figsize=(4, 4))
        plt.axis("off")
        plt.imshow(tile.permute(1, 2, 0).numpy())
        plt.tight_layout()

        plt.savefig(buf1, format="png")
        plt.close()
        buf1.seek(0)

        tmp_path = tmp_folder / f"{floorplan_folder_name}_{i}_original.png"
        Image.open(buf1).save(tmp_path)
        pdf.image(str(tmp_path), x=45, y=30, w=100)

        # ------------ 2nd plot ------------
        buf2 = BytesIO()
        plt.figure(figsize=(12, 5))

        ax1 = plt.subplot(1, 2, 1)
        ax1.set_title("Rooms predictions")
        rseg = ax1.imshow(pol_room_seg, cmap="rooms", vmin=0, vmax=n_rooms - 0.1)
        cbar = plt.colorbar(
            rseg, ticks=np.arange(n_rooms) + 0.5, fraction=0.046, pad=0.01
        )
        cbar.ax.set_yticklabels(room_classes, fontsize=12)
        plt.axis("off")

        ax2 = plt.subplot(1, 2, 2)
        ax2.set_title("Icons predictions")
        iseg = ax2.imshow(pol_icon_seg, cmap="icons", vmin=0, vmax=n_icons - 0.1)
        cbar = plt.colorbar(
            iseg, ticks=np.arange(n_icons) + 0.5, fraction=0.046, pad=0.01
        )
        cbar.ax.set_yticklabels(icon_classes, fontsize=12)
        plt.axis("off")

        plt.savefig(buf2, format="png")
        plt.close()
        buf2.seek(0)

        tmp_path = tmp_folder / f"{floorplan_folder_name}_{i}_preds.png"
        Image.open(buf2).save(tmp_path)
        pdf.image(str(tmp_path), x=0, y=125, w=page_width)

base_name_str = "/ifc_dl/notebooks/building_prototypes_cubicasa5k_model"
threshold_str = f"_T_{THRESHOLD}"
RGB_str = "_RGB"

pdf.output(
    f"{base_name_str}{threshold_str}{RGB_str}.pdf",
    "F",
)
rmtree(tmp_folder)


## Testing the `House` class for loading SVG data

In [ ]:
import cv2
from floortrans.loaders.house import House

png_path = "/ifc_dl/data/cubicasa5k/high_quality/19/F2_scaled.png"
svg_sample_path = "/ifc_dl/data/cubicasa5k/high_quality/19/model.svg"

img = cv2.imread(png_path)

height, width, _ = img.shape
house = House(svg_sample_path, height, width)

house_tensor = house.get_tensor()
house_seg_tensor = house.get_segmentation_tensor()

In [ ]:
plt.imshow(img)
plt.axis("off")
plt.show()

for idx, house_seg_layer in enumerate(house_seg_tensor):
    plt.subplot(1, 2, idx + 1)
    plt.imshow(house_seg_layer)
    plt.axis("off")

plt.show()